In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import math

import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack

import matlab.engine

import utils
from parcels_utils import get_file_info, reload_file_fs

In [ ]:
# x = np.linspace(0,10,50)
# y = 2 * np.sin(x) - 4 * np.cos(x/10)
# variation = 0.5
# y_noisy = y + (np.random.random(y.size) * (variation * 2) - variation)

# s = 7.62
# s = 4.4755

# k = np.arange(1, y_noisy.size + 1)

# lam = 2 - 2 * np.cos((k - 1) * math.pi / k.size)
# gam = 1 / (1 + s * lam ** 2)

# smoothed_y = scipy.fftpack.idctn(gam * scipy.fftpack.dctn(y_noisy, norm="ortho"), norm="ortho")

# plt.plot(x, y, "--")
# plt.plot(x, smoothed_y)
# plt.scatter(x, y_noisy, c="r")
# plt.show()

In [ ]:
eng = matlab.engine.start_matlab()

In [ ]:
target = get_file_info(utils.CURRENT_NETCDF_DIR / "west_coast_1km_hourly/tj_plume_interped.nc", utils.DATA_1KM, name="target")
mask_nc = get_file_info(utils.CURRENT_NETCDF_DIR / "west_coast_1km_hourly/tj_sample.nc", utils.DATA_1KM, name="sample")

In [ ]:
land = utils.generate_mask_none(mask_nc["xrds"]["u"].values)

In [ ]:
target["fs"].U.show()

In [ ]:
u = target["xrds"]["u"].values
v = target["xrds"]["v"].values

In [ ]:
u_list = u[0].tolist()
v_list = v[0].tolist()

In [ ]:
uv_mat = matlab.double([u_list, v_list])

In [ ]:
uv_smooth = eng.smoothn(uv_mat, "robust")

In [ ]:
uv_array = np.empty(uv_smooth.size)
uv_array[:] = uv_smooth
uv_array

In [ ]:
target["xrds"]["u"].values[0] = uv_array[0]
target["xrds"]["v"].values[0] = uv_array[1]

In [ ]:
reload_file_fs(target)
target["fs"].U.show()
target["xrds"]["u"].values[0][np.where(land[0])] = np.nan
target["xrds"]["v"].values[0][np.where(land[0])] = np.nan
reload_file_fs(target)
target["fs"].U.show()

In [ ]:
target["fs"].V.show()